Bpmn2constraints is made to compile BPMN models from both `JSON` and `XML`, but since `JSON` models are proprietary to SAP Signavio, only `XML` support have been added in this repository. Go to [the original repository](https://github.com/signavio/bpmn2constraints) for the full version. 

In order to start using the tool, import the necessary packages.

In [2]:
import os
from Declare4Py.ProcessModels.BpmnConstraintsModel import BpmnConstraintsModel

Some examples are provided in this repository, such as this linear sequence: 
<figure>
    <img width="900" height="200" src="data/linear_diagram.png">
</figure>

With bpmn2constraints, this diagram can be translated into DECLARE constraints, such as:

In [3]:
path = os.path.join("data","linear_sequence.xml")
model = BpmnConstraintsModel()
model.parse_from_file(path)
declare_model = model.declare_model
constraints = declare_model.get_decl_model_constraints()
activities = declare_model.get_model_activities()
print("Model activities:")
print("-----------------")
for idx, act in enumerate(activities):
    print(idx, act)
print("\n")
print("Model constraints:")
print("-----------------")
for idx, con in enumerate(constraints):
    print(idx, con)



Model activities:
-----------------
0 check invoice
1 register invoice
2 accept invoice


Model constraints:
-----------------
0 Init[register invoice] | |
1 Succession[register invoice, check invoice] | |
2 Co-Existence[check invoice, register invoice] | |
3 Choice[check invoice, register invoice] | |
4 Alternate Succession[register invoice, check invoice] | |
5 Succession[check invoice, accept invoice] | |
6 Co-Existence[accept invoice, check invoice] | |
7 Choice[accept invoice, check invoice] | |
8 Alternate Succession[check invoice, accept invoice] | |
9 End[accept invoice] | |


The BpmnConstraintsModel class contains constraints as LTLf as well.

The LTLf formulas are stored in a class containing:
```formula```, ```parsed_formula``` and ```is_satisfiable```

Or the result can be easily printed such as:

In [4]:
for ltl_mod in (model.ltl_model):
    print(ltl_mod.formula)
    print(ltl_mod.parsed_formula)
    print(f'Is satisfiable? {ltl_mod.check_satisfiability()}\n')


con_registerinvoice
con_registerinvoice
Is satisfiable? True

(G((con_registerinvoice) ->  (F(con_checkinvoice)))) & ((F(con_checkinvoice)) | ((~(con_registerinvoice)) & (con_checkinvoice)) | (G(~(con_registerinvoice))))
(and (always (implies con_registerinvoice (eventually con_checkinvoice))) (or (eventually con_checkinvoice) (and (not con_registerinvoice) con_checkinvoice) (always (not con_registerinvoice))))
Is satisfiable? True

(F(con_registerinvoice)) & (F(con_checkinvoice))
(and (eventually con_registerinvoice) (eventually con_checkinvoice))
Is satisfiable? True

(F(con_registerinvoice)) | (F(con_checkinvoice))
(or (eventually con_registerinvoice) (eventually con_checkinvoice))
Is satisfiable? True

(G((con_registerinvoice) ->  (X[!]((F(con_checkinvoice)) | ((~(con_registerinvoice)) & (con_checkinvoice)))))) & (G((con_checkinvoice) ->  ((X[!](~(con_checkinvoice))) & ((F(con_registerinvoice)) | ((~(con_checkinvoice)) & (con_registerinvoice))))))
(and (always (implies con_register

The following blocks are bigger and more complex examples of usage of bpmn2constraints. Based on a bpmn diagram from the Sepsis Cases event log.

<figure>
    <img width="700" height="1000" src="data/Sepsis Cases.png">
</figure>

In [5]:
path = "data/Sepsis Cases.xml" #Path to data 
model = BpmnConstraintsModel()
model.parse_from_file(model_path=path)   

declare_model = model.declare_model
constraints = declare_model.get_decl_model_constraints()
activities = declare_model.get_model_activities()

print("Model activities:")
print("-----------------")
for idx, act in enumerate(activities):
    print(idx, act)
print("\n")
print("Model constraints:")
print("-----------------")
for idx, con in enumerate(constraints):
    print(idx, con)

Model activities:
-----------------
0 CRP
1 IV Antibiotics
2 ER Registration
3 Admission IC
4 Release A
5 ER Sepsis Triage
6 Return ER
7 Admission NC
8 Release D
9 Leucocytes
10 Lactic Acid
11 Release C
12 ER Triage
13 Release B
14 IV Liquid
15 Release E


Model constraints:
-----------------
0 Init[ER Registration] | |
1 Succession[ER Triage, ER Sepsis Triage] | |
2 Co-Existence[ER Sepsis Triage, ER Triage] | |
3 Alternate Succession[ER Triage, ER Sepsis Triage] | |
4 End[Release A] | |
5 End[Release B] | |
6 End[Release C] | |
7 End[Release D] | |
8 End[Release E] | |
9 End[Return ER] | |
10 Co-Existence[CRP, Leucocytes] | |
11 Co-Existence[ER Triage, Leucocytes] | |
12 Co-Existence[Lactic Acid, Leucocytes] | |
13 Co-Existence[ER Triage, CRP] | |
14 Co-Existence[Lactic Acid, CRP] | |
15 Co-Existence[Lactic Acid, ER Triage] | |
16 Precedence[ER Registration, Leucocytes] | |
17 Alternate Precedence[ER Registration, Leucocytes] | |
18 Precedence[ER Registration, CRP] | |
19 Alternate Pr

In [6]:

for ltl_mod in model.ltl_model:
    print(ltl_mod.formula)
    print(ltl_mod.parsed_formula)
    print(f'Is satisfiable? {ltl_mod.check_satisfiability()}\n')


con_erregistration
con_erregistration
Is satisfiable? True

(G((con_ertriage) ->  (F(con_ersepsistriage)))) & ((F(con_ersepsistriage)) | ((~(con_ertriage)) & (con_ersepsistriage)) | (G(~(con_ertriage))))
(and (always (implies con_ertriage (eventually con_ersepsistriage))) (or (eventually con_ersepsistriage) (and (not con_ertriage) con_ersepsistriage) (always (not con_ertriage))))
Is satisfiable? True

(F(con_ertriage)) & (F(con_ersepsistriage))
(and (eventually con_ertriage) (eventually con_ersepsistriage))
Is satisfiable? True

(G((con_ertriage) ->  (X[!]((F(con_ersepsistriage)) | ((~(con_ertriage)) & (con_ersepsistriage)))))) & (G((con_ersepsistriage) ->  ((X[!](~(con_ersepsistriage))) & ((F(con_ertriage)) | ((~(con_ersepsistriage)) & (con_ertriage))))))
(and (always (implies con_ertriage (next (or (eventually con_ersepsistriage) (and (not con_ertriage) con_ersepsistriage))))) (always (implies con_ersepsistriage (and (next (not con_ersepsistriage)) (or (eventually con_ertriage) (and 

Exception: the Lydia command lydia --logic=ltlf --inline=(F(con_releasea)) | (F(con_releasee)) -p failed.
stdout=
stderr=docker: Error response from daemon: failed to create task for container: failed to create shim task: OCI runtime create failed: runc create failed: unable to start container process: unable to apply cgroup configuration: unable to start unit "docker-74e8c5cae5843e6a150e8b54c21be121b58a410c7bf0d7802912aafda863fb96.scope" (properties [{Name:Description Value:"libcontainer container 74e8c5cae5843e6a150e8b54c21be121b58a410c7bf0d7802912aafda863fb96"} {Name:Slice Value:"system.slice"} {Name:Delegate Value:true} {Name:PIDs Value:@au [15115]} {Name:MemoryAccounting Value:true} {Name:CPUAccounting Value:true} {Name:IOAccounting Value:true} {Name:TasksAccounting Value:true} {Name:DefaultDependencies Value:false}]): Message recipient disconnected from message bus without replying: unknown.
time="2024-02-21T14:44:27+01:00" level=error msg="error waiting for container: "


Now that the BpmnConstraintsModel is set up, it can be used to check conformance

In [7]:
from Declare4Py.D4PyEventLog import D4PyEventLog
from Declare4Py.ProcessMiningTasks.ConformanceChecking.LTLAnalyzer import LTLAnalyzer

log_path = os.path.join("../../../../", "tests", "test_logs","Sepsis Cases.xes.gz")
event_log = D4PyEventLog()
event_log.parse_xes_log(log_path)

analyzer = LTLAnalyzer(event_log, model.ltl_model)
conf_check_res_df = analyzer.run_multiple_models(minimize_automaton=False) #This does not seem to work as intented currently
print(f"Accepted traces: {len(conf_check_res_df[conf_check_res_df['accepted'] == True])}")
print(conf_check_res_df)

for idx, ltl_mod in enumerate(model.ltl_model):
    analyzer = LTLAnalyzer(event_log, ltl_mod)
    conf_check_res_df = analyzer.run()
    print(ltl_mod.formula)
    print(f"{idx}. Accepted traces: {len(conf_check_res_df[conf_check_res_df['accepted'] == True])}")



/home/toddy/sap/Declare4Py/.venv/lib/python3.10/site-packages/pm4py/util/dt_parsing/parser.py:76: UserWarning: ISO8601 strings are not fully supported with strpfromiso for Python versions below 3.11
  warnings.warn(
/home/toddy/sap/Declare4Py/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
parsing log, completed traces :: 100%|██████████| 1050/1050 [00:00<00:00, 1381.44it/s]
/home/toddy/sap/Declare4Py/.venv/lib/python3.10/site-packages/pm4py/utils.py:486: UserWarning: the EventLog class has been deprecated and will be removed in a future release.
  warnings.warn("the EventLog class has been deprecated and will be removed in a future release.")


Accepted traces: 0
     case:concept:name  accepted
0                    A     False
1                    B     False
2                    C     False
3                    D     False
4                    E     False
...                ...       ...
1045               HNA     False
1046               INA     False
1047               JNA     False
1048               KNA     False
1049               LNA     False

[1050 rows x 2 columns]
con_erregistration
0. Accepted traces: 995
(G((con_ertriage) ->  (F(con_ersepsistriage)))) & ((F(con_ersepsistriage)) | ((~(con_ertriage)) & (con_ersepsistriage)) | (G(~(con_ertriage))))
1. Accepted traces: 1029
(F(con_ertriage)) & (F(con_ersepsistriage))
2. Accepted traces: 1049
(G((con_ertriage) ->  (X[!]((F(con_ersepsistriage)) | ((~(con_ertriage)) & (con_ersepsistriage)))))) & (G((con_ersepsistriage) ->  ((X[!](~(con_ersepsistriage))) & ((F(con_ertriage)) | ((~(con_ersepsistriage)) & (con_ertriage))))))
3. Accepted traces: 0
F((con_releasea) & (X[!](

Showcasing the issue with DECLARE analyzer

In [8]:
from Declare4Py.ProcessMiningTasks.ConformanceChecking.MPDeclareAnalyzer import MPDeclareAnalyzer, MPDeclareResultsBrowser

basic_checker = MPDeclareAnalyzer(log=event_log, declare_model=declare_model, consider_vacuity=False)
conf_check_res: MPDeclareResultsBrowser = basic_checker.run()

The checker function for template Succession has not been implemented yet.


TypeError: 'NoneType' object is not callable